<a href="https://colab.research.google.com/github/cardazuluaga/DLCourse/blob/main/Ej7PrediccionSeriesTemporalesSenoidal.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.sequence import TimeseriesGenerator

# Genera una serie temporal sintética (por ejemplo, una serie sinusoidal)
N_d = 1000
Nlenght = 100
t = np.linspace(0, Nlenght, N_d)
serie_temporal = np.sin(t) + np.random.normal(0, 0.1, N_d)

# Definición de la ventana temporal deslizante
ventana_temporal = 10  # Tamaño de la ventana
ventana_salida = 1     # Tamaño de la ventana de salida (predicción a un paso)

def preparar_datos(serie_temporal, ventana_temporal):
    x, y = [], []
    for i in range(len(serie_temporal) - ventana_temporal):
        ventana = serie_temporal[i:i + ventana_temporal]
        objetivo = serie_temporal[i + ventana_temporal]
        x.append(ventana)
        y.append(objetivo)
    return np.array(x), np.array(y)

x, y = preparar_datos(serie_temporal, ventana_temporal)

# Divide los datos en conjuntos de entrenamiento y prueba
train_split = 900
x_entrenamiento, x_prueba = x[:train_split], x[train_split:]
y_entrenamiento, y_prueba = y[:train_split], y[train_split:]

modelo_RNN = tf.keras.Sequential([
    tf.keras.layers.SimpleRNN(64, activation='relu', return_sequences=True, input_shape=(ventana_temporal, 1)),
    tf.keras.layers.SimpleRNN(32, activation='relu', return_sequences=True),
    tf.keras.layers.SimpleRNN(32, activation='relu'),
    tf.keras.layers.Dense(1)  # Una sola salida para la predicción
])

# Construye el modelo MLP
modelo = Sequential([
    Dense(64, activation='relu', input_shape=(ventana_temporal,)),
    Dense(32, activation='relu'),
    Dense(32, activation='relu'),
    Dense(ventana_salida)
])

modelo.compile(optimizer=Adam(learning_rate=0.001), loss='mean_squared_error')
modelo_RNN.compile(optimizer=Adam(learning_rate=0.001), loss='mean_squared_error')

# Entrena el modelo
modelo.fit(x_entrenamiento, y_entrenamiento, epochs=100)

historial = modelo_RNN.fit(x_entrenamiento, y_entrenamiento, epochs=100, verbose=1)

# Realiza predicciones en el conjunto de prueba
predicciones = modelo.predict(x_prueba)
predicciones_RNN = modelo_RNN.predict(x_prueba)
predicciones.shape == (Nlenght - ventana_temporal,)
predicciones_RNN.shape == (Nlenght - ventana_temporal,)

In [ ]:
fig, ax = plt.subplots()
ax.plot(y_prueba,'r')
ax.plot(predicciones,'b')
ax.plot(predicciones_RNN,'g')
print('SME_MLP:', np.mean(np.square(y_prueba - predicciones)))
print('SME_RNN:', np.mean(np.square(y_prueba - predicciones_RNN)))